__(a)__ Below you can find the function to convert the images into grayscaled and flattened vectors with the help of Pillow library. Later we will use this function to convert the images in the _3dshapes_train_ and _3dshapes_test_.

In [1]:
import numpy as np
from PIL import Image       
import math

y_train = np.load("orientations_train.npy")

train_size = 10000
image_length = 4096

def vectorize_images(filename, data_size):
    # vectorized 10000 images
    X = np.empty(shape=(data_size, image_length))

    for i in range(data_size):
        img_rgb = Image.open("{filename}/{i}.jpg".format(filename = filename, i = i))
        img_gray = img_rgb.convert('L')

        np_img = np.array(img_gray)
        X[i] = np_img.flatten()
        
    return X

X_train = vectorize_images("3dshapes_train" ,train_size)
    

__(b)__ In order to find the optimal weight vector coefficients, we set the gradient of the least square error function to 0. The closed-form solution we obtain is as follows: <br>

$ \mathbf{w}^* = {({X}^\intercal X)}^{-1}{X}^\intercal \mathbf{t} $ <br>

Below we implemented the function to calculate the optimal weights and used it to train the model. <br> <br>
To test our model we first generated our predictions using the formula: $ \hat{y} = Xw = X{({X}^\intercal X)}^{-1}{X}^\intercal \mathbf{t} $ and then compared it with the actual labels.

In [2]:
# computing the optimal parameters by minimizing the least squares error function
def linear_regression(X, y):
    return np.linalg.pinv(X.T @ X) @ X.T @ y

w = linear_regression(X_train, y_train)

In [5]:
y_test = np.load("orientations_test.npy")

test_size = 1000

X_test = vectorize_images("3dshapes_test" ,test_size)
y_prediction = X_test @ w

RMSE = math.sqrt(np.square(np.subtract(y_test, y_prediction)).mean()) 

print(f"Root Mean Square Error is: {RMSE}")


Root Mean Square Error is: 0.23033949408208762
